In [18]:
import os 
import sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join('..', '..')))

from services.database.database_service import Environment

In [5]:
conn = Environment().get_instance()
db_users = conn.get_db().get('users')
db_daily = conn.get_db().get('daily')

likes_collection = db_users.get_collection("LikesCollection")
wh_collection = db_daily.get_collection("WhatHappenCollection")

2024-05-09 08:31:50,805 INFO root : Environment (__init__): making connection
2024-05-09 08:31:50,808 INFO root : Environment (get_db): access database
2024-05-09 08:31:50,810 INFO root : Environment (get_db): access database


In [ ]:
df = pd.DataFrame.from_records(likes_collection.find({'is_liked': True}))
df

In [28]:
df = pd.DataFrame.from_records(likes_collection.find())

,_id,user_id,happen_id,is_liked,is_disliked,created_at,_class
0,662339556ddd622ac76f9b8e,661f2d5e30ee4c696a1d15fd,65fe4fc0aa58ed17295fa3bf,False,False,2024-04-20 03:41:09.359,innovatenet.com.br.document.LikesDocument
1,662340880077b90b377e4db2,661f2d5e30ee4c696a1d15fd,66208de37b852a12bbe1b311,True,False,2024-04-20 04:11:52.223,innovatenet.com.br.document.LikesDocument
2,662341010077b90b377e4dc3,6621effacf09c02f9a9590d9,66208de37b852a12bbe1b311,True,False,2024-04-20 04:13:53.247,innovatenet.com.br.document.LikesDocument
3,662354fcdeb9562aa7b94876,6621effacf09c02f9a9590d9,66235047cde67915b07344a6,True,False,2024-04-20 05:39:08.173,innovatenet.com.br.document.LikesDocument
4,66235510deb9562aa7b94877,6621effacf09c02f9a9590d9,65fe4fc0aa58ed17295fa3bf,True,False,2024-04-20 05:39:28.174,innovatenet.com.br.document.LikesDocument
5,66235603deb9562aa7b94878,64f74f6e54d0ba6c4b532786,65fe4fc0aa58ed17295fa3bf,True,False,2024-04-20 05:43:31.880,innovatenet.com.br.document.LikesDocument
6,66235666deb9562aa7b94879,64f74f6e54d0ba6c4b532786,66208de37b852a12bbe1b311,True,False,2024-04-20 05:45:10.757,innovatenet.com.br.document.LikesDocument
7,663c4366c4429821f86c3c18,65ae90883470eb010f5f1104,65fe503faa58ed17295fa3c8,True,False,2024-05-09 03:30:46.053,innovatenet.com.br.document.LikesDocument
8,663c43d7c4429821f86c3c1b,65ae90883470eb010f5f1104,65fe52d3aa58ed17295fa3d6,True,False,2024-05-09 03:32:39.192,innovatenet.com.br.document.LikesDocument
9,663c4403c4429821f86c3c1c,661f2d5e30ee4c696a1d15fd,65fe52d3aa58ed17295fa3d6,True,False,2024-05-09 03:33:23.234,innovatenet.com.br.document.LikesDocument


see: https://www.mongodb.com/docs/manual/reference/operator/aggregation/count/


In [11]:
# 
list(likes_collection.aggregate([
    {'$match': {'is_liked': True}},
    {'$group': {'_id': '661f2d5e30ee4c696a1d15fd', 'detail': { '$first': '$$ROOT' },'count': {'$sum': 1}}},
    {'$replaceRoot': { 'newRoot': { '$mergeObjects': [{ 'count': '$count' }, '$detail'] }}},
    ]))

[{'count': 9,
  '_id': ObjectId('662340880077b90b377e4db2'),
  'user_id': '661f2d5e30ee4c696a1d15fd',
  'happen_id': '66208de37b852a12bbe1b311',
  'is_liked': True,
  'is_disliked': False,
  'created_at': datetime.datetime(2024, 4, 20, 4, 11, 52, 223000),
  '_class': 'innovatenet.com.br.document.LikesDocument'}]

In [ ]:
list(likes_collection.aggregate([
    {'$match': {'is_liked': True}},
    {'$group': {'_id': {'user_id': '$$ROOT.user_id'}, 'like_count': {'$sum': 1}} },
    ]))

In [ ]:
# { $count: "passing_scores" }
result = likes_collection.aggregate([
    {'$match': {'is_liked': True, 'user_id': '661f2d5e30ee4c696a1d15fd'}},
    {'$count': "liked_count" },
    ])
list(result)

In [ ]:
list(likes_collection.aggregate([
    {'$match':{'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'}},
    {"$group" :{"_id":"$name", "data" : {"$first":"$$ROOT"}, 'count': {'$sum': 1}}},
    {"$project":{ "happen_id":"$data.happen_id", "user_id" : "$data.user_id", "count" : "$count", "_id" : "$data._id"}}
    ]))

In [ ]:
list(likes_collection.aggregate([
    {'$match':{'$and': [{'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'}]}},
    
    {"$group" :{"_id":"$name", "data" : {"$first":"$$ROOT"}, 'count': {'$sum': 1}}},
    {"$project":{ "happen_id":"$data.happen_id", "user_id" : "$data.user_id", "like_count" : "$count", "_id" : "$data._id"}},

    ]))

see: https://www.mongodb.com/community/forums/t/how-to-use-multiple-conditions-in-match-and-sum-in-group/191477/3


In [ ]:
list(likes_collection.aggregate([
    {'$match': { '$and': [ { 'createdAt': {'$gte': ('2022-09-15'), '$lt':('2022-09-16') }} ]}},
    {'$group':{ '_id': {'createdAt': {'$last': "$createdAt"}}, 'totalAmount': { '$sum': "$postBalance" }}}
    ]))

In [ ]:
list(likes_collection.aggregate([
  {
    '$lookup': {
      'from': "LikesCollection",
      'let': {
        'liked': "$is_liked",
        'disliked': "$is_disliked"
      },
      'pipeline': [
        {
          '$match': {
            '$expr': {
              '$and': [
                {
                  'is_liked': True, 'user_id':'661f2d5e30ee4c696a1d15fd'
                },
                # {
                #   '$eq': [ "$disliked", False]
                # }
              ]
            }
          }
        },
        {
            "$group" : {
                "_id": "$name", 
                "data": { "$first":"$$ROOT" }, 
                'count': {'$sum': 1}}
        },
      ],
      'as': "joined_results"
    }
  },
  {
    '$unwind': { 'path': "$joined_results" }
  }
]))

In [ ]:
pipeline = [
  # first, group ids of buyers and sellers
  {
    '$group': {
      '_id': None,
      'userId': {
        '$addToSet': '$user_id',
      },
      'happenId': {
        '$addToSet': '$happen_id',
      },
    },
  },
  {
      '$group': {
          '_id': None,
          'liked': {
            '$addToSet': '$userId'
          },
        'disliked': {
            '$addToSet': '$is_disliked'
        }
      }
  },
  # then, concat grouped ids into single array 
  # to fetch users with one single $lookup
  # ids in the array will not be unique, 
  # but that does not matter for $lookup
 
  # PS: you can $unwind 'userIsds' 
  # and then $group them like we did with 'like_id' above,
  # if unique values in 'usersIds' bother you :)
  {
    '$project': {
      'userId': {
        '$concatArrays': ['$userId', '$happenId'],
      },
      'liked_count': {
          '$concatArrays': ['$liked']
      },
      'disliked_count': {
          '$concatArrays': ['$disliked']
        }
    },
  },
  # use $lookup to join users by ids from accumulated array
  {
    '$lookup': {
      'from': 'LikesCollection',
      'localField': 'liked_count',
      'foreignField': '_id',
      'as': 'likes',
    },
  },
  # unwind 'users' to be bring out each user to upper level
  # {
  #   '$unwind': '$users',
  # },
  # # make each user a root object in the pipeline array
  # {
  #   '$replaceRoot': {
  #     'newRoot': '$users',
  #   },
  # },
  # sort user objects, like you want
  # {
  #   '$sort': {
  #     'name': 1,
  #   },
  # },
]

list(likes_collection.aggregate(pipeline))

In [ ]:
df = pd.DataFrame.from_records(likes_collection.find({'is_liked': True}), columns=['happen_id', 'user_id', 'is_liked', 'is_disliked'])
df

In [29]:
# https://stackoverflow.com/questions/55097226/mongodb-count-fi-true-in-aggregate-group
list(likes_collection.aggregate([
    {"$match": {"happen_id": "66208de37b852a12bbe1b311"}},
    {"$group": {"_id": "$name", "data": { "$first": "$$ROOT"},
                "liked": { "$sum": {"$cond": [{"$eq": ["$$ROOT.is_liked",True]},1,0]}},
                "disliked": {"$sum": {"$cond": [{"$eq": ["$$ROOT.is_disliked",True]},1,0]}}}},
    {"$project": {"liked_count": "$liked","disliked_count": "$disliked"}},
    {'$unset': ['_id']}]))

[{'liked_count': 3, 'disliked_count': 0}]

In [60]:
list(wh_collection.aggregate([
    {'$match': {'likes': {'$eq': None}}},
    {'$group': {'_id': '$name', 'messges': {'$push': {'$cond': [{'$eq': ['$$ROOT.user_id', '661f2d5e30ee4c696a1d15fd']}, 1, '$$ROOT']}}}},
    {'$unset': ['_id']}
]))

[{'messges': [{'_id': ObjectId('65fe503faa58ed17295fa3c8'),
    'created_at': datetime.datetime(2024, 3, 23, 0, 45, 3, 156000),
    'what_happen': 'mais de quatro horas fazendo um http request devido ao bug não encontrado, quando corrigir a filtro de log então aparece a mensagem de erro e foi menos de um minuto pra corrigir. ',
    'user_id': '65ae90883470eb010f5f1104',
    'name_id': 'beto@daily.com',
    'likes': None,
    'visibility': 'PRIVATE',
    'updated_at': datetime.datetime(2024, 5, 9, 0, 16, 49, 248000)},
   {'_id': ObjectId('65fe52d3aa58ed17295fa3d6'),
    'created_at': datetime.datetime(2024, 3, 23, 0, 56, 3, 526000),
    'what_happen': 'retomando aplicativo de diário ',
    'user_id': '65ae90883470eb010f5f1104',
    'name_id': 'beto@daily.com',
    'likes': None,
    'visibility': 'PRIVATE',
    'updated_at': datetime.datetime(2024, 5, 9, 0, 16, 49, 248000)},
   {'_id': ObjectId('65febf0aaa58ed17295fa3fa'),
    'created_at': datetime.datetime(2024, 3, 23, 8, 37, 46, 5390

In [ ]:
# db.orders.aggregate([
#   {
#     $group: {
#       _id: {
#         country: '$data.nested.country',
#         product: '$data.product',
#       },
#       productCodes: {
#         $push: '$data.latest.sub.code',
#       },
#       uniqueCodes: {
#         $addToSet: '$data.latest.sub.code',
#       }
#     }
#   },
#   {
#     $group: {
#       _id: '$_id.country',
#       country: {
#         $first: '$_id.country',
#       },
#       products: {
#         $push: {
#           product: '$_id.product',
#           codes: {
#             $map: {
#               // run $filter+$size operations per each code
#               input: '$uniqueCodes',
#               as: 'code',
#               in: {
#                 code: '$$code',
#                 count: {
#                   $size: {
#                     $filter: {
#                       // collect same codes into one array
#                       // to be able to count them per product 
#                       input: '$productCodes',
#                       cond: {
#                         $eq: ['$$code', '$$this'],
#                       },
#                     },
#                   },
#                 },
#               },
#             },
#           },
#         },
#       },
#     },
#   },
#   // cleanup
#   {
#     $unset: ['_id'],
#   }
# ]);

[]